In [1]:
from glob import glob
import pandas as pd
import numpy as np

In [2]:
def filter_csvs_gte_2007(csvs):
    '''Filters csvs '''
    return [csv for csv in csvs if int(csv.split('/')[4]) >= 2007]

# Processing Matches

In [3]:
matches_csvs = filter_csvs_gte_2007(glob('../Datasets/frc/events/**/**/*_matches.csv'))

In [4]:
matches_headers=['event_key match_key', 'red1', 'red2', 'red3', 'blue1', 'blue2', 'blue3', 'red_score', 'blue_score']
to_combine = []

for matches_csv in matches_csvs:
    # Read CSV with custom column names
    matches_df = pd.read_csv(matches_csv, header=None, names=matches_headers)
    
    # Separate event_key and matches_key
    matches_df['event_key'], matches_df['match_key'] = zip(*matches_df['event_key match_key'].str.split('_'))
    matches_df.drop(columns=['event_key match_key'], inplace=True)
    
    # Make it a per-team basis
    matches_df = matches_df.melt(
        id_vars=['event_key', 'match_key', 'red_score', 'blue_score'],
        value_vars=['red1', 'red2', 'red3', 'blue1', 'blue2', 'blue3'],
        var_name='matches_role',
        value_name='team',
    )
    
    matches_df['team_score'] = np.where(
        matches_df['matches_role'].str.startswith('red'),
        matches_df['red_score'],
        matches_df['blue_score']
    )
    # matches_df.drop(columns=['red_score', 'blue_score'], inplace=True)
    
    to_combine.append(matches_df)

matches_df = pd.concat(to_combine)
matches_df.head()

,event_key,match_key,red_score,blue_score,matches_role,team,team_score
0,2013azch,f1m1,111.0,69.0,red1,frc3944,111.0
1,2013azch,f1m2,114.0,50.0,red1,frc842,114.0
2,2013azch,qf1m1,87.0,89.0,red1,frc3944,87.0
3,2013azch,qf1m2,106.0,62.0,red1,frc3944,106.0
4,2013azch,qf1m3,133.0,60.0,red1,frc3944,133.0


# Processing Rankings

In [5]:
rankings_csvs = filter_csvs_gte_2007(glob('../Datasets/frc/events/**/**/*_rankings.csv'))

In [6]:
to_combine = []

for rankings_csv in rankings_csvs:
    rankings_df = pd.read_csv(rankings_csv)
    
    rankings_df = rankings_df[['Rank', 'Team']]
    rankings_df.columns = rankings_df.columns.str.lower()
    rankings_df['event_key'] = rankings_csv.split('/')[5]
    
    to_combine.append(rankings_df)

rankings_df = pd.concat(to_combine)
rankings_df.head()

,rank,team,event_key
0,1,1726,2013azch
1,2,1492,2013azch
2,3,1290,2013azch
3,4,2449,2013azch
4,5,60,2013azch


# Processing Awards

In [7]:
awards_csvs = filter_csvs_gte_2007(glob('../Datasets/frc/events/**/**/*_awards.csv'))

In [8]:
to_combine = []

awards_headers=['event_key award_type_enum', 'award_name', 'team', 'awardee']

for awards_csv in awards_csvs:
    awards_df = pd.read_csv(awards_csv, header=None, names=awards_headers)
    
    awards_df['event_key'], awards_df['award_type_enum'] = zip(*awards_df['event_key award_type_enum'].str.split('_', 1))
    
    awards_df['award_type_enum'] = awards_df['award_type_enum'].astype(int)
    
    awards_df['award_type'] = awards_df['award_type_enum'].map({ 0: 'CHAIRMANS', 1: 'WINNER', 2: 'FINALIST' })
    
    awards_df.drop(columns=['awardee', 'event_key award_type_enum'], inplace=True)
    
    to_combine.append(awards_df)

awards_df = pd.concat(to_combine)
awards_df.head()

,award_name,team,event_key,award_type_enum,award_type
0,Regional Chairman's Award,frc2486,2013azch,0,CHAIRMANS
1,Regional Winners,frc3944,2013azch,1,WINNER
2,Regional Winners,frc842,2013azch,1,WINNER
3,Regional Winners,frc1726,2013azch,1,WINNER
4,Rookie All Star Award,frc4629,2013azch,10,NaN
